In [3]:
!git clone https://github.com/SKT-AI/KoBART.git
%cd KoBART
!pip install -r requirements.txt
!pip install .

fatal: destination path 'KoBART' already exists and is not an empty directory.
/Users/noopy/yonsei-exchange-program/KoBART
  Using cached torch-1.7.0-cp38-none-macosx_10_9_x86_64.whl (108.1 MB)
     |████████████████████████████████| 1.4 MB 1.8 MB/s 
Processing /Users/noopy/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
     |████████████████████████████████| 2.0 MB 4.5 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2
You should consider upgrading via the '/Users/noopy/.pyenv/versions/3.8.3/en

In [4]:
!ls

LICENSE          imgs             requirements.txt
README.md        kobart           setup.py


In [9]:
%cd ..

/Users/noopy/yonsei-exchange-program


In [10]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

1.1.0
1.18.5


In [11]:
# pandas 설정: 모든 row와 모든 column을 출력하게 하고, 각 column 별 출력되는 글자수는 50자 이내
pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 50)

In [12]:
# 위의 설정으로 간단하게 보는 df_specific
df_specific = pd.read_csv("./data_wrangled/df_specific_wrangle.csv", encoding="utf-8")
df_specific.head(2)

,대학코드,대학명,gen_info,env_info,food_info,study_info,office_info,facil_info,mhct_info,help_info,etc_info
0,AR000004,Universidad del Salvador,아르헨티나의 대다수 대학교는 한국처럼 캠퍼스가 있는 것이 아니라 도시 내에 몇개의 ...,대학교가 도시 여러곳에 분포하고 있으므로 각기 다르다. 하지만 남녀를 불문하고 부에...,기숙사는 학교에서 제공하지 않고 두가지 옵션이 주어진다. 홈스테이 혹은 게스트 하우...,아르헨티나에서의 대학 수업은 주로 오전 혹은 저녁에 있다. 대부분의 학생들이 일을하...,EL SALVADOR대학교는 국제 교육부가 잘 되있는 학교이다. 아르헨티나 대학교들...,부대시설로는 헬스장이 있다고 알고 있으나 시내와 먼 곳에 있었다. 동아리는 몇가지 ...,딱히 컬쳐쇼크를 받을 만 한 요소는 없었던 것 같다. 하지만 처음에 언어적인 면에서...,"살바도르 대학교 국제처에서 비자문제나 숙소문제(홈스테이경우) 등은 해결해 주고, 특...",NaN
1,AU000019,Australian National University,"ANU has a very big campus, but unlike Yonsei, ...","The city center, called CIVIC, is 20min walk a...","There are many housing options, but I chose to...","In my personal opinions, classes are generally...","Overall, I did not have to ask for help much f...","There were many clubs for you to join at ANU, ...",I did have some culture shock during my first ...,"I was helped a lot, especially during the firs...","Overall, my stay in Australia has been nothing..."


In [13]:
print(df_specific.columns)
print(df_specific.shape)

Index(['대학코드', '대학명', 'gen_info', 'env_info', 'food_info', 'study_info',
       'office_info', 'facil_info', 'mhct_info', 'help_info', 'etc_info'],
      dtype='object')
(470, 11)


In [20]:
# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', None)

In [15]:
import re
# removing other letters except for korean letters from the string data
def remove_junk(str_data):
    str_data = str_data.replace("\\\\n", "")
    str_data = str_data.replace("\\n", "")
    str_data = str_data.replace("\n", "")
    str_data = str_data.replace("\\", "")
    str_data = str_data.replace("\\t", "")
    str_data = str_data.replace("NaN", "")

    # print(str_data) makes an error. too much of data
    # 한글, english 빼고 전부 날리기
    # str_data = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z|0-9|.,]', ' ', str_data)

    # 한글 빼고 다 날리기
    str_data = re.sub('[^가-힣ㄱ-ㅎㅏ]', ' ', str_data)

    # replace multiple spaces into single space
    str_data = ' '.join(str_data.split())
    return str_data

def access_univ_info(dataframe, column, univ_code):
    df_row = dataframe.loc[dataframe["대학코드"] == univ_code]
    str_univ_info = df_row[column].to_string(index=False).lstrip()
    str_univ_info = remove_junk(str_univ_info)
    return str_univ_info


# removing junks only from the string data
def remove_junk_only(str_data):
    str_data = str_data.replace("\\\\n", "")
    str_data = str_data.replace("\\n", "")
    str_data = str_data.replace("\n", "")
    str_data = str_data.replace("\\", "")
    str_data = str_data.replace("\\t", "")
    str_data = str_data.replace("NaN", "")

    # replace multiple spaces into single space
    str_data = ' '.join(str_data.split())
    return str_data

# accessing university info of given column (= accessing one cell)
def access_univ_info_raw(dataframe, column, univ_code):
    df_row = dataframe.loc[dataframe["대학코드"] == univ_code]
    str_univ_info = df_row[column].to_string(index=False).lstrip()
    str_univ_info = remove_junk_only(str_univ_info)
    return str_univ_info

# collecting all column values into one string
def column_to_string(dataframe, column_name: str):
    str_corpus = dataframe[column_name].to_string(index=False).lstrip()
    str_corpus = remove_junk_only(str_corpus)
    return str_corpus

In [16]:
column_data = "gen_info"
univ_code = "DK000003"
str_example = access_univ_info_raw(df_specific, column_data, univ_code)
str_example[:100]

'코펜하겐 대학교는 자연과학캠퍼스, 인문캠퍼스 등 코펜하겐 시내 곳곳에 흩어져 있습니다. 저는 조경건축 수업을 듣기 위해 Frederiksberg 에 위치한 자연과학 캠퍼스도 다니고'

In [21]:
str_example[:1000]

'코펜하겐 대학교는 자연과학캠퍼스, 인문캠퍼스 등 코펜하겐 시내 곳곳에 흩어져 있습니다. 저는 조경건축 수업을 듣기 위해 Frederiksberg 에 위치한 자연과학 캠퍼스도 다니고, 덴마크어 수업과 미디어커뮤니케이션학과 수업을 위해 Amager 에 위치한 캠퍼스에도 다녔습니다. 지하철을 타거나 자전거를 타면 캠퍼스끼리의 거리도 30분 이내라서 돌아다니기 정말 좋았어요. 그리고 캠퍼스 내부 디자인이 정말 깔끔하고 좋았고, 이렇게 멋진 곳이라면 저절로 학교를 다니고 공부하고 싶을 것 같았습니다. 코펜하겐 시내 자체는 그렇게 크지 않아서 자전거를 타거나 걸어다녀도 모든 게 범위 안에 있어서 좋았습니다. 일단 건물들이 6층 이하로 낮아서 탁 트인 하늘이 보이고, 개인적인 취향일 수는 있지만 오래된 건물도 철거하지 않고 조금씩 고쳐서 모습을 간직하고 있는 것이 인상깊었습니다. 그리고 다른 유럽 나라들을 여행할 때도 굉장히 편리했어요. 유럽의 많은 나라와 직항 저가항공이 있고 시간도 ... 한국의 일반적인 대학들과 달리 코펜하겐 대학교는 도시 곳곳에 소규모로 캠퍼스가 여러 군데 있어요. 저는 1학기에는 주로 사회과학대 수업이 열리는 City Campus (CSS)에서 수업을 들었고, 2학기에는 주로 인문대 수업이 열리는 South Campus (KUA)에서 수업을 들었습니다. CSS는 도심에 있는 교통 요충지 Nørreport역 근처에 있고, KUA는 최근에 개발된 Amager 지역의 Islands Brygge 역 근처에 있어요. 그리고 코펜하겐 도시 자체가 굉장히 작기 때문에 왠만한 데는 대중교통이나 자전거로 30-40분 내에 갈 수 있는 거 같아요. 공항도 Nørreport역에서 20분 걸리기 때문에 여행다니기에도 좋습니다.t기후는… 악명이 높다시피 춥고 비가 많이 내립니다 ㅠㅠ 사실 겨울에도 한국에 비하면 기온이 그리 낮은 편은 아닌데 (1년 동안 영하로 내려가는 걸 거의 못 봤어요) 여름이 지나면 해를 거의 못 보고 비도 많이 내려서 진짜 추워요. 바람... 저

In [17]:
# collecting all column values into one string
corpus = column_to_string(df_specific, column_data)
print("글자수:", len(corpus))
corpus[:100]

글자수: 4648952


'아르헨티나의 대다수 대학교는 한국처럼 캠퍼스가 있는 것이 아니라 도시 내에 몇개의 건물이 대학교 건물이라서 수업에 맞는 건물로 수업을 들으러 가면 된다. 도시의 대중교통이 꽤 잘 '